In [159]:
import pandas as pd
import numpy as np
import matplotlib as plt
path = '../Podaci/9_S/2022/zones_20_year_2022.csv'
zone = pd.read_csv(path, low_memory=False)

In [161]:
zone['timestamp'] = pd.to_datetime(zone['timestamp'])
piv = zone.pivot_table(index='timestamp', 
                     columns='zone_id', 
                     values='zone_fan_speed', 
                     aggfunc='first')

piv = piv.sort_index()    #sortiramo po vremenu
piv = piv.bfill()         #backfill jer je svaki drugi podatak za par tisucinki pomaknuti timestamp s NaN vrijednostima


changes_any_zone = piv.diff().ne(0).any(axis=1)     
#print(changes_any_zone)

#svaka promjena je novi id
piv['segment_id'] = changes_any_zone.cumsum()

segments = piv.groupby('segment_id', as_index=False).agg(
    start_time = ('segment_id', lambda x: x.index[0]),
    end_time   = ('segment_id', lambda x: x.index[-1])
)

# Izračunamo trajanje segmenta
segments['duration'] = segments['end_time'] - segments['start_time']

segments_15min = segments[segments['duration'] >= pd.Timedelta(minutes=15)]

segments_15min.to_csv("segmenti_bez_promjena_fan_speeda.csv", index=False)